In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install filterpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177 kB 5.0 MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110474 sha256=4c686f089ecb3b4f76ff486d71a75c26c7f5b9ff76aeaa112ba8eb29bd5a055f
  Stored in directory: /root/.cache/pip/wheels/fe/f6/cb/40331472edf4fd399b8cad02973c6acbdf26898342928327fe
Successfully built filterpy


In [1]:
!pip install Autogluon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 289 kB 4.6 MB/s 
     |████████████████████████████████| 49 kB 834 kB/s 
     |████████████████████████████████| 62 kB 54 kB/s 
     |████████████████████████████████| 286 kB 18.7 MB/s 
     |████████████████████████████████| 226 kB 54.5 MB/s 
     |████████████████████████████████| 59 kB 1.2 MB/s 
     |████████████████████████████████| 103 kB 3.4 MB/s 
     |████████████████████████████████| 41 kB 95 kB/s 
     |████████████████████████████████| 1.0 MB 39.2 MB/s 
     |████████████████████████████████| 802 kB 45.4 MB/s 
     |████████████████████████████████| 132 kB 50.5 MB/s 
     |████████████████████████████████| 60.2 MB 1.1 MB/s 
     |████████████████████████████████| 1.6 MB 58.5 MB/s 
     |████████████████████████████████| 280 kB 49.0 MB/s 
     |████████████████████████████████| 74 kB 3.8 MB/s 
     |████████████████████████████████| 43 kB

In [29]:
import pandas as pd
import random
import os
import numpy as np
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import math
from scipy.signal import butter, lfilter
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
import itertools
import math

# V1

In [30]:
train = pd.read_csv('/content/drive/MyDrive/감귤/train.csv')
test_x = pd.read_csv('/content/drive/MyDrive/감귤/test.csv')

In [31]:
train.drop(['ID'],axis = 1, inplace=True)
test_x.drop(['ID'],axis = 1, inplace=True)

In [32]:
train.drop(train.filter(regex = '엽록소').columns, axis = 1, inplace=True)
test_x.drop(test_x.filter(regex = '엽록소').columns, axis = 1, inplace=True)

In [33]:
train['newsoon_ju'] = 0
train['newsoon_ju'][train['2022-09-01 새순'] < 3] = 0.0213
train['newsoon_ju'][(train['2022-09-01 새순'] >= 3) & (train['2022-09-01 새순'] < 3.5)] = 0.0275
train['newsoon_ju'][(train['2022-09-01 새순'] >= 3.5) & (train['2022-09-01 새순'] < 4)] = 0.0335
train['newsoon_ju'][(train['2022-09-01 새순'] >= 4) & (train['2022-09-01 새순'] < 4.5)] = 0.0395
train['newsoon_ju'][train['2022-09-01 새순'] >= 4.5] = 0.0461


test_x['newsoon_ju'] = 0
test_x['newsoon_ju'][test_x['2022-09-01 새순'] < 3] = 0.0213
test_x['newsoon_ju'][(test_x['2022-09-01 새순'] >= 3) & (test_x['2022-09-01 새순'] < 3.5)] = 0.0275
test_x['newsoon_ju'][(test_x['2022-09-01 새순'] >= 3.5) & (test_x['2022-09-01 새순'] < 4)] = 0.0335
test_x['newsoon_ju'][(test_x['2022-09-01 새순'] >= 4) & (test_x['2022-09-01 새순'] < 4.5)] = 0.0395
test_x['newsoon_ju'][test_x['2022-09-01 새순'] >= 4.5] = 0.0461
test_x['newsoon_ju']

<ipython-input-33-c911370097a6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['newsoon_ju'][train['2022-09-01 새순'] < 3] = 0.0213
<ipython-input-33-c911370097a6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['newsoon_ju'][(train['2022-09-01 새순'] >= 3) & (train['2022-09-01 새순'] < 3.5)] = 0.0275
<ipython-input-33-c911370097a6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['newsoon_ju'][(train['2022-09-01 새순'] >=

0       0.0395
1       0.0213
2       0.0461
3       0.0335
4       0.0213
         ...  
2203    0.0213
2204    0.0395
2205    0.0335
2206    0.0461
2207    0.0461
Name: newsoon_ju, Length: 2208, dtype: float64

In [34]:
new_soon_tr = train.filter(regex = '새순')
new_soon_tr = new_soon_tr[sorted(new_soon_tr.columns)]
for i in range(len(new_soon_tr)):
  for j in range(len(new_soon_tr.columns) - 1):
    if(new_soon_tr.iloc[i,j] < new_soon_tr.iloc[i,j+1]):
      new_soon_tr.iloc[i,j+1] = new_soon_tr.iloc[i,j]
    if((new_soon_tr.iloc[i,j] == new_soon_tr.iloc[i,j+1]) | (new_soon_tr.iloc[i,j+1] == 0)):
      new_soon_tr.iloc[i,j+1] = new_soon_tr.iloc[i,j] - train.iloc[i,-1]
new_soon_tr[new_soon_tr < 0] = 0
new_soon_tr


new_soon_te = test_x.filter(regex = '새순')
for i in range(len(new_soon_te)):
  for j in range(len(new_soon_te.columns) - 1):
    if(new_soon_te.iloc[i,j] < new_soon_te.iloc[i,j+1]): 
      new_soon_te.iloc[i,j+1] = new_soon_te.iloc[i,j]
    if((new_soon_te.iloc[i,j] == new_soon_te.iloc[i,j+1]) | (new_soon_te.iloc[i,j+1] == 0)):
      new_soon_te.iloc[i,j+1] = new_soon_te.iloc[i,j] - test_x.iloc[i,-1]
new_soon_te[new_soon_te < 0] = 0
new_soon_te

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
<ipython-input-34-fc3935f9c134>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

,2022-09-01 새순,2022-09-02 새순,2022-09-03 새순,2022-09-04 새순,2022-09-05 새순,2022-09-06 새순,2022-09-07 새순,2022-09-08 새순,2022-09-09 새순,2022-09-10 새순,...,2022-11-19 새순,2022-11-20 새순,2022-11-21 새순,2022-11-22 새순,2022-11-23 새순,2022-11-24 새순,2022-11-25 새순,2022-11-26 새순,2022-11-27 새순,2022-11-28 새순
0,4.3,4.2000,4.1605,4.1000,4.0605,4.0210,4.0000,3.9605,3.9000,3.8605,...,0.8000,0.7605,0.7000,0.6605,0.6000,0.5605,0.5210,0.5000,0.4605,0.4210
1,2.5,2.4787,2.4574,2.4361,2.4148,2.4000,2.3787,2.3574,2.3361,2.3148,...,0.7787,0.7574,0.7361,0.7148,0.7000,0.6787,0.6574,0.6361,0.6000,0.5787
2,4.7,4.6539,4.6000,4.5539,4.5000,4.4539,4.4000,4.3539,4.3000,4.2539,...,0.8539,0.8000,0.7539,0.7000,0.6539,0.6000,0.5539,0.5000,0.4539,0.4078
3,3.6,3.5665,3.5330,3.4995,3.4660,3.4325,3.4000,3.3665,3.3330,3.3000,...,0.8000,0.7665,0.7000,0.6665,0.6330,0.6000,0.5665,0.5330,0.5000,0.4665
4,2.7,2.6787,2.6574,2.6361,2.6000,2.5787,2.5574,2.5361,2.5000,2.4787,...,0.8361,0.8000,0.7787,0.7574,0.7361,0.7000,0.6787,0.6574,0.6361,0.6148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203,2.5,2.4787,2.4000,2.3787,2.3574,2.3361,2.3000,2.2787,2.2574,2.2361,...,0.7509,0.7296,0.7083,0.7000,0.6787,0.6574,0.6361,0.6000,0.5787,0.5574
2204,4.1,4.0605,4.0210,4.0000,3.9605,3.9000,3.8605,3.8000,3.7605,3.7000,...,0.8000,0.7605,0.7000,0.6605,0.6000,0.5605,0.5210,0.5000,0.4605,0.4210
2205,3.8,3.7665,3.7330,3.7000,3.6665,3.6330,3.6000,3.5665,3.5000,3.4665,...,0.9000,0.8665,0.8000,0.7665,0.7330,0.7000,0.6665,0.6330,0.6000,0.5665
2206,4.5,4.4000,4.3539,4.3000,4.2539,4.2000,4.1539,4.1078,4.1000,4.0539,...,0.8539,0.8000,0.7539,0.7000,0.6539,0.6078,0.6000,0.5000,0.4539,0.4078


In [35]:
new_soon_tr = new_soon_tr.add_suffix('_new')
new_soon_te = new_soon_te.add_suffix('_new')
train = pd.concat([train,new_soon_tr],axis=1)

test_x = pd.concat([test_x,new_soon_te],axis=1)

In [36]:
train_data = TabularDataset(train)
test_data = TabularDataset(test_x)
print("==================Tabular_complete========================")
print("==================Tabular_complete========================")
print("==================Tabular_complete========================")
print("==================Tabular_complete========================")
print("==================Tabular_complete========================")
predictor = TabularPredictor(label='착과량(int)',  eval_metric='mean_absolute_error').fit(train_data, presets='high_quality',  ag_args_fit={'num_gpus': 0})
print("==================learning_complete========================")
print("==================learning_complete========================")
print("==================learning_complete========================")
print("==================learning_complete========================")
print("==================learning_complete========================")
print("==================learning_complete========================")
y_pred = predictor.predict(test_data)
print("==================predictor_complete========================")
print("==================predictor_complete========================")
print("==================predictor_complete========================")
print("==================predictor_complete========================")
print("==================predictor_complete========================")
y_pred = pd.DataFrame(y_pred, columns=['착과량(int)'])
V1 = pd.read_csv('/content/drive/MyDrive/감귤/sample_submission.csv')
V1['착과량(int)'] = y_pred
V1.to_csv('V1.csv', index=False)
print("==================submission_complete========================")
print("==================submission_complete========================")
print("==================submission_complete========================")
print("==================submission_complete========================")
print("==================submission_complete========================")
print("==================submission_complete========================")

No path specified. Models will be saved in: "AutogluonModels/ag-20221214_133301/"
Presets specified: ['high_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20221214_133301/"
AutoGluon Version:  0.6.1
Python Version:     3.8.16
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Aug 26 08:44:51 UTC 2022
Train Data Rows:    2207
Train Data Columns: 183
Label Column: 착과량(int)
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == int and many unique label-values observed).
	Label info (max, min, mean, stddev): (799, 1, 406.22247, 218.9783)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators 

==================Tabular_complete========================
==================Tabular_complete========================
==================Tabular_complete========================
==================Tabular_complete========================
==================Tabular_complete========================


	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 183 | ['수고(m)', '수관폭1(min)', '수관폭2(max)', '수관폭평균', '2022-09-01 새순', ...]
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 183 | ['수고(m)', '수관폭1(min)', '수관폭2(max)', '수관폭평균', '2022-09-01 새순', ...]
	0.2s = Fit runtime
	183 features in original data used to generate 183 features in processed data.
	Train Data (Processed) Memory Usage: 3.23 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.31s ...
AutoGluon will gauge predictive performance using evaluation metric: 'mean_absolute_error'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.
	To change this, specify the eval_metric pa

==================learning_complete========================
==================learning_complete========================
==================learning_complete========================
==================learning_complete========================
==================learning_complete========================
==================learning_complete========================
==================predictor_complete========================
==================predictor_complete========================
==================predictor_complete========================
==================predictor_complete========================
==================predictor_complete========================
==================submission_complete========================
==================submission_complete========================
==================submission_complete========================
==================submission_complete========================
==================submission_complete========================
==================submiss

In [37]:
train.to_csv('V1train.csv',index=False)

# V2

In [38]:
train = pd.read_csv('/content/drive/MyDrive/감귤/train.csv')
test_x = pd.read_csv('/content/drive/MyDrive/감귤/test.csv')

In [39]:
train.drop(['ID'],axis = 1, inplace=True)
test_x.drop(['ID'],axis = 1, inplace=True)

In [40]:
train.drop(train.filter(regex = '엽록소').columns, axis = 1, inplace=True)
test_x.drop(test_x.filter(regex = '엽록소').columns, axis = 1, inplace=True)

In [41]:
def LPF(series, low, order=1):
    b, a = butter(
                  N = order,
                  Wn = low,
                  btype = 'low',
                  )
    lpf_series = lfilter(b, a, series)
    
    return lpf_series

In [42]:
train_lpf = train.filter(regex='새순')
train_lpf = train_lpf.T
train_lpf = train_lpf.add_suffix('_lpf')
train_merge = LPF(train_lpf,0.1,1)
train_merge = train_merge.T
train_merge = pd.DataFrame(train_merge)
train_merge = train_merge.add_suffix('_lpf')
train = pd.concat([train, train_merge],axis=1)


test_lpf = test_x.filter(regex='새순')
test_lpf = test_lpf.T
test_lpf = test_lpf.add_suffix('_lpf')
test_merge = LPF(test_lpf,0.1,1)
test_merge = test_merge.T
test_merge = pd.DataFrame(test_merge)
test_merge = test_merge.add_suffix('_lpf')
test_x = pd.concat([test_x, test_merge],axis=1)

In [43]:
import os
import torch
import random
import numpy as np
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate, train_test_split

train_data = TabularDataset(train)
test_data = TabularDataset(test_x)
print("==================Tabular_complete========================")
print("==================Tabular_complete========================")
print("==================Tabular_complete========================")
print("==================Tabular_complete========================")
print("==================Tabular_complete========================")
predictor = TabularPredictor(label='착과량(int)',  eval_metric='mean_absolute_error').fit(train_data, presets='high_quality',  ag_args_fit={'num_gpus': 0})
print("==================learning_complete========================")
print("==================learning_complete========================")
print("==================learning_complete========================")
print("==================learning_complete========================")
print("==================learning_complete========================")
print("==================learning_complete========================")
y_pred = predictor.predict(test_data)
print("==================predictor_complete========================")
print("==================predictor_complete========================")
print("==================predictor_complete========================")
print("==================predictor_complete========================")
print("==================predictor_complete========================")
y_pred = pd.DataFrame(y_pred, columns=['착과량(int)'])
V2 = pd.read_csv('/content/drive/MyDrive/감귤/sample_submission.csv')
V2['착과량(int)'] = y_pred
V2.to_csv('V2.csv', index=False)
print("==================submission_complete========================")
print("==================submission_complete========================")
print("==================submission_complete========================")
print("==================submission_complete========================")
print("==================submission_complete========================")
print("==================submission_complete========================")

No path specified. Models will be saved in: "AutogluonModels/ag-20221214_133414/"
Presets specified: ['high_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20221214_133414/"
AutoGluon Version:  0.6.1
Python Version:     3.8.16
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Aug 26 08:44:51 UTC 2022
Train Data Rows:    2207
Train Data Columns: 182
Label Column: 착과량(int)
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == int and many unique label-values observed).
	Label info (max, min, mean, stddev): (799, 1, 406.22247, 218.9783)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators 

==================Tabular_complete========================
==================Tabular_complete========================
==================Tabular_complete========================
==================Tabular_complete========================
==================Tabular_complete========================


	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 182 | ['수고(m)', '수관폭1(min)', '수관폭2(max)', '수관폭평균', '2022-09-01 새순', ...]
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 182 | ['수고(m)', '수관폭1(min)', '수관폭2(max)', '수관폭평균', '2022-09-01 새순', ...]
	0.2s = Fit runtime
	182 features in original data used to generate 182 features in processed data.
	Train Data (Processed) Memory Usage: 3.21 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.33s ...
AutoGluon will gauge predictive performance using evaluation metric: 'mean_absolute_error'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.
	To change this, specify the eval_metric pa

==================learning_complete========================
==================learning_complete========================
==================learning_complete========================
==================learning_complete========================
==================learning_complete========================
==================learning_complete========================
==================predictor_complete========================
==================predictor_complete========================
==================predictor_complete========================
==================predictor_complete========================
==================predictor_complete========================
==================submission_complete========================
==================submission_complete========================
==================submission_complete========================
==================submission_complete========================
==================submission_complete========================
==================submiss

In [44]:
train.to_csv('V2train.csv',index=False)

# Ens

In [45]:
ens = pd.read_csv('/content/drive/MyDrive/감귤/sample_submission.csv')
ens['착과량(int)'] = (V1['착과량(int)'] + V2['착과량(int)']) / 2
ens['착과량(int)'] = round(ens['착과량(int)'])
ens.to_csv('submit.csv',index=False)